## C S 329E HW 9

# Crossvalidation and hyperparameter selection

Tori Garfield teg755
Veronica Alejandro vaa678

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [44]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
import sklearn
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes


# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [45]:
(df_X, s_y) = load_diabetes(return_X_y=True, as_frame=True)

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [46]:
def get_linear_regression_model( df_X, s_y ):
    
    # turn y into a column vector
#     s_y = s_y[:, np.newaxis]
    
    beta_hat = np.linalg.lstsq(df_X, s_y, rcond=-1)[0]
    return beta_hat
    

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [47]:
def partition_data( df_X, s_y, k ):
    dict_k_df_X = {}
    dict_k_s_y = {}
    indices_break = np.linspace(0, len(df_X), k+1, dtype = int)
    for i in range(k):
        dict_k_df_X[i+1] = df_X[indices_break[i]:indices_break[i+1]]
        dict_k_s_y[i+1] = s_y[indices_break[i]:indices_break[i+1]]
    return (dict_k_df_X, dict_k_s_y)

In [48]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )


In [49]:

sumFolds = 0

for key, values in dict_k_df_X.items():
    sumFolds += len(values)
    print("Fold " + str(key) + ": Dataframe size: " + str(len(values)) + " Series length: " + str(len(dict_k_s_y[key])))
    
print("Element size in each fold: " + str(sumFolds) + " Rows in original dataframe " + str(len(df_X)))


Fold 1: Dataframe size: 88 Series length: 88
Fold 2: Dataframe size: 88 Series length: 88
Fold 3: Dataframe size: 89 Series length: 89
Fold 4: Dataframe size: 88 Series length: 88
Fold 5: Dataframe size: 89 Series length: 89
Element size in each fold: 442 Rows in original dataframe 442


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [50]:
def get_mae( s_y, s_y_hat):
    
    mae = np.sum(np.abs(s_y - s_y_hat) / len(s_y))
            
    return mae 


In [51]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [52]:
mae = np.array([])

for k in dict_k_df_X.keys():
    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype = float)
    
    for notK in dict_k_df_X.keys():
        if notK != k:
            train_df_X = pd.concat([train_df_X, dict_k_df_X[notK]])
            train_s_y = train_s_y.append(dict_k_s_y[notK])
            
    train_df_X = pd.concat([pd.DataFrame({'intercept': np.ones(len(train_df_X))}, index = train_df_X.index), train_df_X], axis = 1)        
    test_df_X = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
    test_df_X = pd.concat([pd.DataFrame({'intercept': np.ones(len(test_df_X))}, index = test_df_X.index), test_df_X], axis = 1)        

    beta_hat2 = get_linear_regression_model(train_df_X, train_s_y)
    s_y_hat = test_df_X.apply(lambda x : np.matmul(x, beta_hat2), axis = 1)
  
    mae = np.append( mae, get_mae(test_s_y,s_y_hat) ) 

In [53]:
print('Minimum MAE: {}'.format(mae.min()))
print('Maximum MAE: {}'.format(mae.max()))
print('Mean MAE: {}'.format(mae.mean()))


Minimum MAE: 42.54819305780144
Maximum MAE: 47.93278393155872
Mean MAE: 44.27849621230541


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [54]:
iris = load_iris()
df_X = iris['data']
s_y = iris['target']

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [55]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)

sumFolds = 0

for key, values in dict_k_df_X.items():
    sumFolds += len(values)
    print("Fold " + str(key) + ": Dataframe size: " + str(len(values)) + ", Series length: " + str(len(dict_k_s_y[key])))
    
print("Element size in each fold: " + str(sumFolds) + "\nRows in original dataframe: " + str(len(df_X)))



Fold 1: Dataframe size: 30, Series length: 30
Fold 2: Dataframe size: 30, Series length: 30
Fold 3: Dataframe size: 30, Series length: 30
Fold 4: Dataframe size: 30, Series length: 30
Fold 5: Dataframe size: 30, Series length: 30
Element size in each fold: 150
Rows in original dataframe: 150


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [56]:
def get_acc(s_1, s_2):
    length = len(s_1)
    sumEqual = 0.0;
    for i in range(length):
        if s_1[i] == s_2[i]:
            sumEqual += 1
    return sumEqual / length     


In [57]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [58]:

# Outer loop
outer_acc = np.array([])
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])
for k in dict_k_df_X.keys():

    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype = float)

    avg_acc = np.array([])
    
    for pos_min_impurity in possible_min_impurity_decrease:
        
        imp_acc = np.array([])
        print('Testing {:.2f} min impurity decrease'.format(pos_min_impurity))

        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        innerKeys = list(dict_k_df_X.keys())
        innerKeys.remove(k) # fold does not include k

        for innerK in innerKeys:
          inner_train_df_X = pd.DataFrame()
          inner_train_s_y = pd.Series(dtype = float)

          for notInnerK in innerKeys:
            if notInnerK != innerK:
              df =  pd.DataFrame(dict_k_df_X[notInnerK])
              series = pd.Series(dict_k_s_y[notInnerK])
              inner_train_df_X = pd.concat([inner_train_df_X, df])
              inner_train_s_y = inner_train_s_y.append(series) 

          inner_test_df_X = dict_k_df_X[innerK] 
          inner_test_s_y = dict_k_s_y[innerK]   

          # inner decision tree
          clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = pos_min_impurity)
          clf = clf.fit(inner_train_df_X, inner_train_s_y)

          accInner = get_acc(inner_test_s_y, clf.predict(inner_test_df_X))
          imp_acc = np.append(imp_acc, accInner)

        avg_acc = np.append(avg_acc, imp_acc.mean()) # append mean accuracy
        print("\tAverage accuracy over 4 folds is " + str(np.round(imp_acc.mean(), 2)))

    # Use best min impurity decrease to train model
    idxBest = np.argmin(1 - avg_acc)
    bestMinImpDec = possible_min_impurity_decrease[idxBest]
    print('\nBest min impurity decrease is {}\n'.format(bestMinImpDec))
    
    # outer accuracy calculation 
    for notK in dict_k_df_X.keys():
      if notK != k:
        df2 = pd.DataFrame(dict_k_df_X[notK])
        train_df_X = pd.concat([train_df_X, df2])
        series2 = pd.Series(dict_k_s_y[notK])
        train_s_y = train_s_y.append(series2)
    test_df_X = dict_k_df_X[k] 
    test_s_y = dict_k_s_y[k]
    # outer decision tree
    clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = bestMinImpDec)
    clf = clf.fit(train_df_X, train_s_y)
    this_acc = get_acc(test_s_y, clf.predict(test_df_X))
    outer_acc = np.append(outer_acc, this_acc) # make sure and calculate this_acc in your loop

Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.73
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.52
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.17
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.08

Best min impurity decrease is 0.1

Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.58
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.23
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.08
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.0

Best min impurity decrease is 0.1

Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.9
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.58
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.58
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.17

Best min impurity decrease is 0.1

Testin

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [59]:
print('Min accuracy: {}'.format(np.round(outer_acc.min(), 2)))
print('Max accuracy: {}'.format(np.round(outer_acc.max(), 2)))
print('Mean accuracy: {}'.format(np.round(outer_acc.mean(), 2)))

Min accuracy: 0.0
Max accuracy: 1.0
Mean accuracy: 0.75
